# Lab 4: Offset Management and Exactly-Once Processing

## 🎯 **Learning Objectives:**
- Understand Kafka offset management mechanisms
- Learn about different offset reset strategies
- Practice manual offset commits for reliability
- Explore exactly-once processing patterns
- Handle consumer failures and recovery scenarios

## 📚 **Key Concepts:**
1. **Offset Management**: Tracking message consumption progress
2. **Auto vs Manual Commit**: Trade-offs between convenience and reliability
3. **Exactly-Once Processing**: Ensuring no duplicate or lost messages
4. **Consumer Failure Recovery**: Handling crashes and restarts
5. **Offset Reset Strategies**: earliest, latest, none

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Producer      │───▶│   Kafka Topic    │───▶│   Consumer      │
│                 │    │   (3 Partitions) │    │   Group         │
└─────────────────┘    └──────────────────┘    └─────────────────┘
         │                        │                        │
         ▼                        ▼                        ▼
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│ Transactional   │    │ Partition 0      │    │ Offset Tracking │
│ Producer        │    │ Partition 1      │    │ • Auto Commit   │
│                 │    │ Partition 2      │    │ • Manual Commit │
│                 │    │                  │    │ • Exactly-Once  │
└─────────────────┘    └──────────────────┘    └─────────────────┘
```


In [1]:
# Install and Import Dependencies
import json
import random
import time
import threading
from datetime import datetime
from collections import defaultdict
from kafka import KafkaProducer, KafkaConsumer, KafkaAdminClient
from kafka.errors import KafkaError, CommitFailedError
from kafka.admin import ConfigResource, ConfigResourceType
from kafka.structs import TopicPartition
import uuid

print("✅ All dependencies imported successfully!")


✅ All dependencies imported successfully!


In [2]:
# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
TOPIC_NAME = 'stock-data-offsets'
CONSUMER_GROUP = 'offset-management-group'

# Offset Management Strategies
OFFSET_STRATEGIES = {
    'auto_commit': 'Automatic offset commits (default)',
    'manual_commit': 'Manual offset commits for reliability',
    'exactly_once': 'Exactly-once processing with transactions',
    'offset_reset': 'Different offset reset strategies'
}

print("🔧 Kafka Configuration:")
print(f"   Bootstrap Servers: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"   Topic: {TOPIC_NAME}")
print(f"   Consumer Group: {CONSUMER_GROUP}")
print(f"   Available Strategies: {list(OFFSET_STRATEGIES.keys())}")


🔧 Kafka Configuration:
   Bootstrap Servers: localhost:9092
   Topic: stock-data-offsets
   Consumer Group: offset-management-group
   Available Strategies: ['auto_commit', 'manual_commit', 'exactly_once', 'offset_reset']


In [3]:
# Create Topic for Offset Management Lab
def create_topic_if_not_exists(topic_name: str, num_partitions: int = 3):
    """Create topic if it doesn't exist"""
    try:
        admin_client = KafkaAdminClient(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            client_id='offset_lab_admin'
        )
        
        # Check if topic exists
        metadata = admin_client.describe_cluster()
        print(f"✅ Connected to Kafka cluster")
        
        # Try to get topic metadata
        try:
            topic_metadata = admin_client.describe_topics([topic_name])
            topic_info = topic_metadata[topic_name]
            existing_partitions = len(topic_info.partitions)
            
            if existing_partitions == num_partitions:
                print(f"✅ Topic '{topic_name}' already exists with {num_partitions} partitions")
            else:
                print(f"⚠️ Topic '{topic_name}' exists but has {existing_partitions} partitions, need {num_partitions}")
                print(f"🔄 Deleting and recreating topic...")
                
                # Delete existing topic
                admin_client.delete_topics([topic_name])
                time.sleep(2)  # Wait for deletion
                
                # Create new topic with correct partitions
                from kafka.admin import NewTopic
                topic = NewTopic(
                    name=topic_name,
                    num_partitions=num_partitions,
                    replication_factor=1
                )
                admin_client.create_topics([topic])
                print(f"✅ Topic '{topic_name}' recreated with {num_partitions} partitions")
                
        except Exception as e:
            print(f"📝 Creating topic '{topic_name}' with {num_partitions} partitions...")
            from kafka.admin import NewTopic
            topic = NewTopic(
                name=topic_name,
                num_partitions=num_partitions,
                replication_factor=1
            )
            admin_client.create_topics([topic])
            print(f"✅ Topic '{topic_name}' created successfully")
        
        admin_client.close()
        
    except Exception as e:
        print(f"❌ Error managing topic: {e}")
        print("💡 Make sure Kafka is running: docker-compose up -d")

# Create the topic
create_topic_if_not_exists(TOPIC_NAME, 3)


✅ Connected to Kafka cluster
📝 Creating topic 'stock-data-offsets' with 3 partitions...
✅ Topic 'stock-data-offsets' created successfully


In [4]:
# Stock Data Producer for Offset Management
class StockDataProducer:
    def __init__(self, bootstrap_servers: str, topic: str):
        self.topic = topic
        self.symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', 'META', 'NVDA', 'NFLX', 'ADBE', 'CRM']
        self.base_prices = {
            'AAPL': 150.0, 'GOOGL': 2800.0, 'MSFT': 350.0, 'TSLA': 250.0, 'AMZN': 3200.0,
            'META': 300.0, 'NVDA': 450.0, 'NFLX': 400.0, 'ADBE': 500.0, 'CRM': 200.0
        }
    
    def generate_ohlcv_data(self, symbol: str, base_price: float) -> dict:
        """Generate realistic OHLCV data"""
        price_change = random.uniform(-0.02, 0.02)
        new_price = base_price * (1 + price_change)
        
        open_price = round(new_price * random.uniform(0.998, 1.002), 2)
        close_price = round(new_price * random.uniform(0.998, 1.002), 2)
        high_price = round(max(open_price, close_price) * random.uniform(1.001, 1.005), 2)
        low_price = round(min(open_price, close_price) * random.uniform(0.995, 0.999), 2)
        
        volume = random.randint(100000, 1000000)
        
        return {
            "symbol": symbol,
            "timestamp": datetime.now().isoformat() + "Z",
            "open": open_price,
            "high": high_price,
            "low": low_price,
            "close": close_price,
            "volume": volume,
            "exchange": "NASDAQ",
            "message_id": str(uuid.uuid4())  # Unique message ID for tracking
        }
    
    def send_test_data(self, num_messages: int = 50):
        """Send test data for offset management experiments"""
        print(f"📈 Generating {num_messages} test messages...")
        
        producer_config = {
            'bootstrap_servers': KAFKA_BOOTSTRAP_SERVERS,
            'value_serializer': lambda v: json.dumps(v).encode('utf-8'),
            'key_serializer': lambda k: k.encode('utf-8') if k else None,
            'acks': 'all',  # Wait for all replicas
            'retries': 3,
            'retry_backoff_ms': 100
        }
        
        producer = KafkaProducer(**producer_config)
        
        for i in range(num_messages):
            symbol = random.choice(self.symbols)
            base_price = self.base_prices[symbol]
            ohlcv_data = self.generate_ohlcv_data(symbol, base_price)
            
            # Send message
            future = producer.send(
                self.topic, 
                key=symbol, 
                value=ohlcv_data
            )
            
            try:
                record_metadata = future.get(timeout=10)
                print(f"📊 Sent {symbol}: ${ohlcv_data['close']} -> Partition {record_metadata.partition}, Offset {record_metadata.offset}")
            except KafkaError as e:
                print(f"❌ Error sending message: {e}")
            
            time.sleep(0.05)  # Small delay between messages
        
        producer.flush()
        producer.close()
        
        print(f"✅ Successfully sent {num_messages} test messages")

# Initialize producer
producer = StockDataProducer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME)
print("✅ Stock Data Producer initialized!")


✅ Stock Data Producer initialized!


In [ ]:
# Offset Management Consumer Classes
class OffsetManagementConsumer:
    """Base class for offset management consumers"""
    
    def __init__(self, bootstrap_servers: str, topic: str, group_id: str):
        self.bootstrap_servers = bootstrap_servers
        self.topic = topic
        self.group_id = group_id
        self.consumer = None
        self.processed_messages = []
        self.failed_messages = []
    
    def create_consumer(self, **kwargs):
        """Create consumer with specified configuration"""
        default_config = {
            'bootstrap_servers': self.bootstrap_servers,
            'group_id': self.group_id,
            'value_deserializer': lambda m: json.loads(m.decode('utf-8')),
            'key_deserializer': lambda m: m.decode('utf-8') if m else None,
            'auto_offset_reset': 'earliest',
            'enable_auto_commit': True,
            'auto_commit_interval_ms': 1000,
            'session_timeout_ms': 30000,
            'heartbeat_interval_ms': 10000,
            'consumer_timeout_ms': 1000  # Add consumer timeout
        }
        
        # Merge with provided config
        config = {**default_config, **kwargs}
        self.consumer = KafkaConsumer(self.topic, **config)
        return self.consumer
    
    def process_message(self, message):
        """Process a single message - override in subclasses"""
        data = message.value
        print(f"📊 Processing {data['symbol']}: ${data['close']} (Message ID: {data['message_id']})")
        
        # Simulate processing time
        time.sleep(0.1)
        
        # Simulate occasional failures (10% chance)
        if random.random() < 0.1:
            raise Exception(f"Simulated processing failure for message {data['message_id']}")
        
        return data
    
    def consume_messages(self, max_messages: int = 10, timeout_ms: int = 5000):
        """Consume messages with offset management"""
        if not self.consumer:
            raise Exception("Consumer not created. Call create_consumer() first.")
        
        print(f"🔄 Starting to consume up to {max_messages} messages...")
        
        message_count = 0
        start_time = time.time()
        
        try:
            for message in self.consumer:
                if message_count >= max_messages:
                    break
                
                try:
                    processed_data = self.process_message(message)
                    self.processed_messages.append({
                        'message_id': processed_data['message_id'],
                        'symbol': processed_data['symbol'],
                        'partition': message.partition,
                        'offset': message.offset,
                        'timestamp': processed_data['timestamp']
                    })
                    print(f"✅ Processed message {message_count + 1}/{max_messages}")
                    
                except Exception as e:
                    print(f"❌ Failed to process message: {e}")
                    self.failed_messages.append({
                        'message_id': message.value.get('message_id', 'unknown'),
                        'error': str(e),
                        'partition': message.partition,
                        'offset': message.offset
                    })
                
                message_count += 1
                
                # Check timeout
                if (time.time() - start_time) * 1000 > timeout_ms:
                    print(f"⏰ Timeout reached after {timeout_ms}ms")
                    break
                    
        except Exception as e:
            print(f"❌ Consumer error: {e}")
        finally:
            if self.consumer:
                self.consumer.close()
        
        print(f"📊 Consumption completed: {len(self.processed_messages)} processed, {len(self.failed_messages)} failed")
        return self.processed_messages, self.failed_messages

class AutoCommitConsumer(OffsetManagementConsumer):
    """Consumer with automatic offset commits"""
    
    def create_consumer(self, **kwargs):
        config = {
            'enable_auto_commit': True,
            'auto_commit_interval_ms': 1000,
            **kwargs
        }
        return super().create_consumer(**config)

class ManualCommitConsumer(OffsetManagementConsumer):
    """Consumer with manual offset commits"""
    
    def create_consumer(self, **kwargs):
        config = {
            'enable_auto_commit': False,
            **kwargs
        }
        return super().create_consumer(**config)
    
    def consume_messages(self, max_messages: int = 10, commit_every: int = 5, timeout_ms: int = 5000):
        """Consume messages with manual commits"""
        if not self.consumer:
            raise Exception("Consumer not created. Call create_consumer() first.")
        
        print(f"🔄 Starting manual commit consumer (commit every {commit_every} messages)...")
        
        message_count = 0
        start_time = time.time()
        
        try:
            for message in self.consumer:
                if message_count >= max_messages:
                    break
                
                try:
                    processed_data = self.process_message(message)
                    self.processed_messages.append({
                        'message_id': processed_data['message_id'],
                        'symbol': processed_data['symbol'],
                        'partition': message.partition,
                        'offset': message.offset,
                        'timestamp': processed_data['timestamp']
                    })
                    print(f"✅ Processed message {message_count + 1}/{max_messages}")
                    
                    # Manual commit every N messages
                    if (message_count + 1) % commit_every == 0:
                        try:
                            self.consumer.commit()
                            print(f"💾 Committed offsets after {message_count + 1} messages")
                        except CommitFailedError as e:
                            print(f"❌ Commit failed: {e}")
                    
                except Exception as e:
                    print(f"❌ Failed to process message: {e}")
                    self.failed_messages.append({
                        'message_id': message.value.get('message_id', 'unknown'),
                        'error': str(e),
                        'partition': message.partition,
                        'offset': message.offset
                    })
                
                message_count += 1
                
                # Check timeout
                if (time.time() - start_time) * 1000 > timeout_ms:
                    print(f"⏰ Timeout reached after {timeout_ms}ms")
                    break
            
            # Final commit
            try:
                self.consumer.commit()
                print(f"💾 Final commit completed")
            except CommitFailedError as e:
                print(f"❌ Final commit failed: {e}")
                    
        except Exception as e:
            print(f"❌ Consumer error: {e}")
        finally:
            if self.consumer:
                self.consumer.close()
        
        print(f"📊 Manual commit consumption completed: {len(self.processed_messages)} processed, {len(self.failed_messages)} failed")
        return self.processed_messages, self.failed_messages

print("📋 Available consumer types:")
print("   - AutoCommitConsumer: Automatic offset commits")
print("   - ManualCommitConsumer: Manual offset commits")
print("   - OffsetManagementConsumer: Base class for custom implementations")


📋 Available consumer types:
   - AutoCommitConsumer: Automatic offset commits
   - ManualCommitConsumer: Manual offset commits
   - OffsetManagementConsumer: Base class for custom implementations


## Exercise 1: Auto Commit vs Manual Commit

### 🎯 **Learning Objectives:**
- Understand automatic offset commits
- Learn manual offset commit patterns
- Compare reliability between approaches
- Observe offset behavior during failures

### 📚 **Key Concepts:**
1. **Auto Commit**: Convenient but less reliable
2. **Manual Commit**: More control and reliability
3. **Commit Timing**: When to commit offsets
4. **Failure Recovery**: Handling processing failures


In [6]:
# Exercise 1: Auto Commit vs Manual Commit Demo
print("🔄 Exercise 1: Comparing Auto Commit vs Manual Commit")

# First, send some test data
print("\n📈 Step 1: Sending test data...")
producer.send_test_data(num_messages=20)

print("\n🤖 Step 2: Testing Auto Commit Consumer...")
# Test Auto Commit Consumer
auto_consumer = AutoCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-auto")
auto_consumer.create_consumer()
auto_processed, auto_failed = auto_consumer.consume_messages(max_messages=10)

print(f"\n📊 Auto Commit Results:")
print(f"   ✅ Processed: {len(auto_processed)} messages")
print(f"   ❌ Failed: {len(auto_failed)} messages")

print("\n🤖 Step 3: Testing Manual Commit Consumer...")
# Test Manual Commit Consumer
manual_consumer = ManualCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-manual")
manual_consumer.create_consumer()
manual_processed, manual_failed = manual_consumer.consume_messages(max_messages=10, commit_every=3)

print(f"\n📊 Manual Commit Results:")
print(f"   ✅ Processed: {len(manual_processed)} messages")
print(f"   ❌ Failed: {len(manual_failed)} messages")

print("\n📋 Comparison Analysis:")
print("✅ Auto Commit: Convenient, commits automatically every 1 second")
print("✅ Manual Commit: More control, commits after processing batches")
print("⚠️ Auto Commit: May lose messages if consumer crashes before commit")
print("⚠️ Manual Commit: Requires careful error handling to avoid duplicates")


🔄 Exercise 1: Comparing Auto Commit vs Manual Commit

📈 Step 1: Sending test data...
📈 Generating 20 test messages...
📊 Sent MSFT: $356.81 -> Partition 0, Offset 0
📊 Sent TSLA: $247.4 -> Partition 2, Offset 0
📊 Sent MSFT: $356.34 -> Partition 0, Offset 1
📊 Sent NFLX: $403.32 -> Partition 1, Offset 0
📊 Sent AMZN: $3255.12 -> Partition 2, Offset 1
📊 Sent NFLX: $395.45 -> Partition 1, Offset 1
📊 Sent NVDA: $448.39 -> Partition 2, Offset 2
📊 Sent AAPL: $147.64 -> Partition 0, Offset 2
📊 Sent NFLX: $401.05 -> Partition 1, Offset 2
📊 Sent GOOGL: $2769.15 -> Partition 1, Offset 3
📊 Sent CRM: $197.76 -> Partition 1, Offset 4
📊 Sent NFLX: $406.86 -> Partition 1, Offset 5
📊 Sent TSLA: $246.87 -> Partition 2, Offset 3
📊 Sent AMZN: $3141.19 -> Partition 2, Offset 4
📊 Sent GOOGL: $2790.82 -> Partition 1, Offset 6
📊 Sent NVDA: $455.58 -> Partition 2, Offset 5
📊 Sent MSFT: $344.87 -> Partition 0, Offset 3
📊 Sent AMZN: $3147.97 -> Partition 2, Offset 6
📊 Sent NFLX: $393.7 -> Partition 1, Offset 7
📊 Se

## Exercise 2: Offset Reset Strategies

### 🎯 **Learning Objectives:**
- Understand different offset reset strategies
- Learn when to use each strategy
- Practice offset management for new consumer groups
- Handle scenarios with no committed offsets

### 📚 **Key Concepts:**
1. **earliest**: Start from the beginning of the topic
2. **latest**: Start from the end (newest messages)
3. **none**: Fail if no committed offset exists
4. **Consumer Group**: Offset management per group


In [8]:
# Exercise 2: Offset Reset Strategies Demo
print("🔄 Exercise 2: Testing Different Offset Reset Strategies")

# Send more test data
print("\n📈 Step 1: Sending additional test data...")
producer.send_test_data(num_messages=15)

print("\n🔍 Step 2: Testing 'earliest' offset reset...")
# Test earliest offset reset
earliest_consumer = AutoCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-earliest")
earliest_consumer.create_consumer(auto_offset_reset='earliest')
earliest_processed, earliest_failed = earliest_consumer.consume_messages(max_messages=5)

print(f"\n📊 Earliest Reset Results:")
print(f"   ✅ Processed: {len(earliest_processed)} messages")
if earliest_processed:
    print(f"   📍 First message offset: {earliest_processed[0]['offset']}")

print("\n🔍 Step 3: Testing 'latest' offset reset...")
# Test latest offset reset - send data first, then consume
print("   📈 Sending data for latest consumer...")
producer.send_test_data(num_messages=10)

latest_consumer = AutoCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-latest")
latest_consumer.create_consumer(auto_offset_reset='latest')
latest_processed, latest_failed = latest_consumer.consume_messages(max_messages=5, timeout_ms=3000)

print(f"\n📊 Latest Reset Results:")
print(f"   ✅ Processed: {len(latest_processed)} messages")
if latest_processed:
    print(f"   📍 First message offset: {latest_processed[0]['offset']}")

print("\n🔍 Step 4: Testing 'none' offset reset...")
# Test none offset reset
try:
    none_consumer = AutoCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-none")
    none_consumer.create_consumer(auto_offset_reset='none')
    none_processed, none_failed = none_consumer.consume_messages(max_messages=5, timeout_ms=2000)
    
    print(f"\n📊 None Reset Results:")
    print(f"   ✅ Processed: {len(none_processed)} messages")
except Exception as e:
    print(f"\n❌ None Reset Error: {e}")
    print("   This is expected behavior - 'none' fails when no committed offset exists")

print("\n📋 Offset Reset Strategy Analysis:")
print("✅ earliest: Good for replaying all data, processing historical messages")
print("✅ latest: Good for real-time processing, only new messages")
print("✅ none: Good for strict offset management, fails if no committed offset")
print("⚠️ Choose strategy based on your use case and data requirements")


🔄 Exercise 2: Testing Different Offset Reset Strategies

📈 Step 1: Sending additional test data...
📈 Generating 15 test messages...
📊 Sent NVDA: $446.28 -> Partition 2, Offset 10
📊 Sent AAPL: $150.68 -> Partition 0, Offset 12
📊 Sent ADBE: $493.13 -> Partition 2, Offset 11
📊 Sent GOOGL: $2761.89 -> Partition 1, Offset 13
📊 Sent AMZN: $3152.21 -> Partition 2, Offset 12
📊 Sent TSLA: $245.75 -> Partition 2, Offset 13
📊 Sent AMZN: $3216.9 -> Partition 2, Offset 14
📊 Sent GOOGL: $2755.93 -> Partition 1, Offset 14
📊 Sent META: $297.37 -> Partition 0, Offset 13
📊 Sent MSFT: $356.03 -> Partition 0, Offset 14
📊 Sent TSLA: $252.74 -> Partition 2, Offset 15
📊 Sent GOOGL: $2771.82 -> Partition 1, Offset 15
📊 Sent NVDA: $446.58 -> Partition 2, Offset 16
📊 Sent AMZN: $3237.48 -> Partition 2, Offset 17
📊 Sent CRM: $199.77 -> Partition 1, Offset 16
✅ Successfully sent 15 test messages

🔍 Step 2: Testing 'earliest' offset reset...
🔄 Starting to consume up to 5 messages...
📊 Processing MSFT: $353.76 (Mes

## Exercise 3: Consumer Failure and Recovery

### 🎯 **Learning Objectives:**
- Simulate consumer failures
- Understand offset behavior during crashes
- Practice recovery scenarios
- Learn about message duplication and loss

### 📚 **Key Concepts:**
1. **Consumer Failure**: What happens when consumer crashes
2. **Offset Recovery**: How offsets are managed during failures
3. **Message Duplication**: Risk of processing same message twice
4. **Message Loss**: Risk of losing uncommitted messages


In [9]:
# Exercise 3: Consumer Failure and Recovery Demo
print("🔄 Exercise 3: Simulating Consumer Failures and Recovery")

# Send test data for failure simulation
print("\n📈 Step 1: Sending test data for failure simulation...")
producer.send_test_data(num_messages=20)

print("\n💥 Step 2: Simulating consumer failure with auto commit...")
# Simulate failure with auto commit
class FailingAutoConsumer(AutoCommitConsumer):
    def process_message(self, message):
        data = message.value
        print(f"📊 Processing {data['symbol']}: ${data['close']} (Message ID: {data['message_id']})")
        
        # Simulate processing time
        time.sleep(0.1)
        
        # Fail after processing 3 messages
        if len(self.processed_messages) >= 2:
            raise Exception(f"Simulated consumer crash after processing {len(self.processed_messages)} messages")
        
        return data

# Test failing auto commit consumer
failing_auto = FailingAutoConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-failing-auto")
failing_auto.create_consumer()
try:
    auto_failed_processed, auto_failed_failed = failing_auto.consume_messages(max_messages=10)
except Exception as e:
    print(f"💥 Consumer crashed: {e}")
    auto_failed_processed = failing_auto.processed_messages
    auto_failed_failed = failing_auto.failed_messages

print(f"\n📊 Auto Commit Failure Results:")
print(f"   ✅ Processed before crash: {len(auto_failed_processed)} messages")
print(f"   ❌ Failed: {len(auto_failed_failed)} messages")

print("\n🔄 Step 3: Testing recovery with same consumer group...")
# Test recovery with same consumer group
recovery_consumer = AutoCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-failing-auto")
recovery_consumer.create_consumer()
recovery_processed, recovery_failed = recovery_consumer.consume_messages(max_messages=10)

print(f"\n📊 Recovery Results:")
print(f"   ✅ Processed after recovery: {len(recovery_processed)} messages")
print(f"   ❌ Failed: {len(recovery_failed)} messages")

print("\n💥 Step 4: Simulating consumer failure with manual commit...")
# Simulate failure with manual commit
class FailingManualConsumer(ManualCommitConsumer):
    def process_message(self, message):
        data = message.value
        print(f"📊 Processing {data['symbol']}: ${data['close']} (Message ID: {data['message_id']})")
        
        # Simulate processing time
        time.sleep(0.1)
        
        # Fail after processing 2 messages (before commit)
        if len(self.processed_messages) >= 1:
            raise Exception(f"Simulated consumer crash before commit")
        
        return data

# Test failing manual commit consumer
failing_manual = FailingManualConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-failing-manual")
failing_manual.create_consumer()
try:
    manual_failed_processed, manual_failed_failed = failing_manual.consume_messages(max_messages=10, commit_every=3)
except Exception as e:
    print(f"💥 Consumer crashed: {e}")
    manual_failed_processed = failing_manual.processed_messages
    manual_failed_failed = failing_manual.failed_messages

print(f"\n📊 Manual Commit Failure Results:")
print(f"   ✅ Processed before crash: {len(manual_failed_processed)} messages")
print(f"   ❌ Failed: {len(manual_failed_failed)} messages")

print("\n🔄 Step 5: Testing recovery with manual commit consumer...")
# Test recovery with manual commit consumer
manual_recovery_consumer = ManualCommitConsumer(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME, f"{CONSUMER_GROUP}-failing-manual")
manual_recovery_consumer.create_consumer()
manual_recovery_processed, manual_recovery_failed = manual_recovery_consumer.consume_messages(max_messages=10, commit_every=3)

print(f"\n📊 Manual Recovery Results:")
print(f"   ✅ Processed after recovery: {len(manual_recovery_processed)} messages")
print(f"   ❌ Failed: {len(manual_recovery_failed)} messages")

print("\n📋 Failure and Recovery Analysis:")
print("✅ Auto Commit: May lose messages if crash happens before commit interval")
print("✅ Manual Commit: More reliable, commits only after successful processing")
print("⚠️ Consumer failures can lead to message duplication or loss")
print("⚠️ Recovery behavior depends on commit strategy and timing")


🔄 Exercise 3: Simulating Consumer Failures and Recovery

📈 Step 1: Sending test data for failure simulation...
📈 Generating 20 test messages...
📊 Sent CRM: $202.24 -> Partition 1, Offset 19
📊 Sent META: $299.35 -> Partition 0, Offset 18
📊 Sent ADBE: $503.99 -> Partition 2, Offset 23
📊 Sent NVDA: $445.72 -> Partition 2, Offset 24
📊 Sent NFLX: $404.99 -> Partition 1, Offset 20
📊 Sent GOOGL: $2809.24 -> Partition 1, Offset 21
📊 Sent GOOGL: $2808.14 -> Partition 1, Offset 22
📊 Sent CRM: $199.93 -> Partition 1, Offset 23
📊 Sent META: $297.5 -> Partition 0, Offset 19
📊 Sent GOOGL: $2830.9 -> Partition 1, Offset 24
📊 Sent AMZN: $3247.57 -> Partition 2, Offset 25
📊 Sent NVDA: $441.95 -> Partition 2, Offset 26
📊 Sent NVDA: $441.17 -> Partition 2, Offset 27
📊 Sent NVDA: $444.48 -> Partition 2, Offset 28
📊 Sent META: $294.48 -> Partition 0, Offset 20
📊 Sent GOOGL: $2764.3 -> Partition 1, Offset 25
📊 Sent CRM: $202.65 -> Partition 1, Offset 26
📊 Sent TSLA: $247.08 -> Partition 2, Offset 29
📊 Sent 

## Exercise 4: Best Practices and Cleanup

### 🎯 **Learning Objectives:**
- Learn offset management best practices
- Understand when to use each approach
- Review key takeaways and recommendations
- Clean up resources and consumer groups

### 📚 **Best Practices:**
1. **Choose the right commit strategy** for your use case
2. **Handle failures gracefully** with proper error handling
3. **Monitor consumer lag** to detect processing issues
4. **Use idempotent processing** to handle duplicates safely


In [10]:
# Exercise 4: Best Practices and Cleanup
print("📚 Offset Management Best Practices and Recommendations")

print("\n🎯 When to Use Each Commit Strategy:")
print("\n1. 🔄 Auto Commit:")
print("   ✅ Use when: Processing is fast and failures are rare")
print("   ✅ Use when: Message loss is acceptable")
print("   ✅ Use when: Simplicity is more important than reliability")
print("   ❌ Avoid when: Processing is slow or complex")
print("   ❌ Avoid when: Message loss is not acceptable")

print("\n2. 💾 Manual Commit:")
print("   ✅ Use when: Processing is slow or complex")
print("   ✅ Use when: Message loss is not acceptable")
print("   ✅ Use when: You need exactly-once processing")
print("   ✅ Use when: You need fine-grained control")
print("   ❌ Avoid when: Processing is very simple and fast")

print("\n3. 🔄 Offset Reset Strategies:")
print("   ✅ earliest: Replay all data, historical processing")
print("   ✅ latest: Real-time processing, only new messages")
print("   ✅ none: Strict offset management, fail if no committed offset")

print("\n📋 General Best Practices:")
print("✅ Always handle consumer failures gracefully")
print("✅ Use idempotent processing to handle duplicates")
print("✅ Monitor consumer lag to detect processing issues")
print("✅ Commit offsets after successful processing")
print("✅ Use appropriate timeout settings")
print("✅ Test failure scenarios thoroughly")
print("✅ Document your offset management strategy")

print("\n🎯 Key Takeaways:")
print("1. Offset management is crucial for reliable message processing")
print("2. Choose commit strategy based on your reliability requirements")
print("3. Manual commits provide better control but require more care")
print("4. Consumer failures can lead to message duplication or loss")
print("5. Test failure scenarios to understand recovery behavior")
print("6. Monitor and alert on consumer lag and processing failures")

print("\n🧹 Cleanup:")
print("✅ Consumer groups will be cleaned up automatically")
print("✅ Offsets are managed by Kafka automatically")
print("✅ No manual cleanup required for this lab")

print("\n✅ Lab 4: Offset Management and Exactly-Once Processing completed!")
print("🚀 Next: Try Lab 5: Real-time Analytics or explore advanced Kafka features")


📚 Offset Management Best Practices and Recommendations

🎯 When to Use Each Commit Strategy:

1. 🔄 Auto Commit:
   ✅ Use when: Processing is fast and failures are rare
   ✅ Use when: Message loss is acceptable
   ✅ Use when: Simplicity is more important than reliability
   ❌ Avoid when: Processing is slow or complex
   ❌ Avoid when: Message loss is not acceptable

2. 💾 Manual Commit:
   ✅ Use when: Processing is slow or complex
   ✅ Use when: Message loss is not acceptable
   ✅ Use when: You need exactly-once processing
   ✅ Use when: You need fine-grained control
   ❌ Avoid when: Processing is very simple and fast

3. 🔄 Offset Reset Strategies:
   ✅ earliest: Replay all data, historical processing
   ✅ latest: Real-time processing, only new messages
   ✅ none: Strict offset management, fail if no committed offset

📋 General Best Practices:
✅ Always handle consumer failures gracefully
✅ Use idempotent processing to handle duplicates
✅ Monitor consumer lag to detect processing issues
✅ C